# Day 9: ResNet - Deep Residual Learning 🏗️

Welcome to Day 9 of 30 Papers in 30 Days!

Today we're exploring **ResNet (Residual Networks)** - the architecture that solved the degradation problem and enabled training of networks with 100+ layers. In 2015, ResNet didn't just win ImageNet - it proved that deeper networks could actually be better when built correctly.

## What You'll Learn

1. **The Degradation Problem**: Why deeper networks performed worse
2. **Skip Connections**: The elegant solution that changed everything
3. **Residual Learning**: Learning the difference instead of the mapping
4. **Implementation**: Building ResNet blocks from scratch
5. **Gradient Flow**: How skip connections solve vanishing gradients
6. **Modern Impact**: Why every architecture now uses skip connections

## The Big Idea (in 30 seconds)

**Problem**: Deeper networks should be better, but they performed WORSE (even on training data!)

**ResNet Solution**: Add skip connections - `output = F(x) + x`

**Result**: Networks can now be 100+ layers deep and actually improve with depth!

**Magic**: Skip connections create "information highways" that preserve gradient flow

Let's dive into the skip connection revolution! 🚀

In [ ]:
# Setup and imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

# Add current directory to path
sys.path.append('.')

# Import our ResNet implementation
from implementation import ResNet18, ResNet50, BasicBlock, BottleneckBlock
from visualization import ResNetVisualizer
from train_minimal import ResNetTrainer, create_synthetic_dataset

# Set up device and seeds
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)

print(f"🔥 Using device: {device}")
print("✅ All imports successful!")
print("🎯 Ready to explore residual learning!")

## Part 1: Understanding the Degradation Problem

Before ResNet, researchers discovered something puzzling: deeper networks performed WORSE than shallow ones, even on training data! This wasn't overfitting - it was a fundamental optimization problem.

Let's demonstrate this degradation problem ourselves.

In [ ]:
# Demonstrate the degradation problem
def demonstrate_degradation_problem():
    """Show that deeper plain networks perform worse."""
    
    print("🔬 Demonstrating the Degradation Problem...")
    
    # Create plain networks (no skip connections) of different depths
    class PlainNet(nn.Module):
        def __init__(self, depth, num_classes=10):
            super().__init__()
            layers = []
            in_channels = 3
            
            for i in range(depth):
                out_channels = 64 if i < depth//2 else 128
                layers.append(nn.Conv2d(in_channels, out_channels, 3, 1, 1))
                layers.append(nn.BatchNorm2d(out_channels))
                layers.append(nn.ReLU(inplace=True))
                in_channels = out_channels
            
            layers.append(nn.AdaptiveAvgPool2d((1, 1)))
            self.features = nn.Sequential(*layers)
            self.fc = nn.Linear(128, num_classes)
            
        def forward(self, x):
            x = self.features(x)
            x = torch.flatten(x, 1)
            return self.fc(x)
    
    # Test different depths
    depths = [10, 20, 30, 40]
    results = {}
    
    # Create simple dataset
    print("\n📦 Creating synthetic dataset...")
    X = torch.randn(1000, 3, 32, 32)
    y = torch.randint(0, 10, (1000,))
    dataset = torch.utils.data.TensorDataset(X, y)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
    
    for depth in depths:
        print(f"\n🏋️ Training {depth}-layer plain network...")
        model = PlainNet(depth=depth).to(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        criterion = nn.CrossEntropyLoss()
        
        # Train for a few epochs
        losses = []
        for epoch in range(5):
            epoch_loss = 0
            for batch_x, batch_y in dataloader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
            
            avg_loss = epoch_loss / len(dataloader)
            losses.append(avg_loss)
        
        results[depth] = losses
        print(f"  Final loss: {losses[-1]:.4f}")
    
    # Plot results
    plt.figure(figsize=(10, 6))
    for depth, losses in results.items():
        plt.plot(losses, label=f'{depth} layers', linewidth=2, marker='o')
    
    plt.xlabel('Epoch')
    plt.ylabel('Training Loss')
    plt.title('Degradation Problem: Deeper Networks Perform Worse!')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("\n💡 Key Insight:")
    print("  Notice how deeper networks have HIGHER training loss!")
    print("  This isn't overfitting - it's an optimization problem")
    print("  Skip connections (ResNet) solve this!")

demonstrate_degradation_problem()

## Part 2: The ResNet Solution - Skip Connections

ResNet's brilliant insight: instead of learning `H(x)` directly, learn the residual `F(x) = H(x) - x`, then add it back: `H(x) = F(x) + x`

### Why This Works

**Identity Mapping**: If the optimal function is the identity (do nothing), the network just needs to set `F(x) = 0`, which is much easier than learning `H(x) = x` directly.

**Gradient Flow**: Skip connections create a direct path for gradients to flow backward, preventing vanishing gradients.

Let's build and visualize residual blocks!

In [ ]:
# Explore ResNet building blocks
def explore_residual_blocks():
    """Understand how residual blocks work."""
    
    print("🏗️ Exploring Residual Blocks...")
    
    # Create a basic residual block
    block = BasicBlock(inplanes=64, planes=64, stride=1)
    
    print("\n📐 Basic Residual Block Structure:")
    print(block)
    
    # Forward pass to see dimensions
    input_tensor = torch.randn(1, 64, 32, 32)
    
    print(f"\nInput shape: {list(input_tensor.shape)}")
    
    # Manually trace through the block
    identity = input_tensor
    
    # First conv path
    out = block.conv1(input_tensor)
    out = block.bn1(out)
    out = F.relu(out)
    print(f"After conv1-bn1-relu: {list(out.shape)}")
    
    # Second conv path
    out = block.conv2(out)
    out = block.bn2(out)
    print(f"After conv2-bn2: {list(out.shape)}")
    
    # Add skip connection
    out += identity
    print(f"After adding skip connection: {list(out.shape)}")
    
    # Final activation
    out = F.relu(out)
    print(f"Final output: {list(out.shape)}")
    
    # Visualize the block
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Block diagram
    ax1.text(0.5, 0.9, 'Input x', ha='center', fontsize=12, weight='bold')
    ax1.arrow(0.5, 0.85, 0, -0.1, head_width=0.05, head_length=0.03, fc='black')
    
    # Main path
    ax1.add_patch(plt.Rectangle((0.3, 0.55), 0.4, 0.15, fill=True, facecolor='lightblue', edgecolor='black'))
    ax1.text(0.5, 0.625, 'Conv-BN-ReLU', ha='center', fontsize=10)
    ax1.arrow(0.5, 0.55, 0, -0.05, head_width=0.05, head_length=0.02, fc='black')
    
    ax1.add_patch(plt.Rectangle((0.3, 0.35), 0.4, 0.15, fill=True, facecolor='lightblue', edgecolor='black'))
    ax1.text(0.5, 0.425, 'Conv-BN', ha='center', fontsize=10)
    ax1.arrow(0.5, 0.35, 0, -0.05, head_width=0.05, head_length=0.02, fc='black')
    
    # Skip connection
    ax1.arrow(0.15, 0.9, 0, -0.55, head_width=0.05, head_length=0.02, fc='red', ec='red', linestyle='--', linewidth=2)
    ax1.text(0.08, 0.65, 'Skip', ha='center', fontsize=10, color='red', weight='bold')
    
    # Addition
    ax1.add_patch(plt.Circle((0.5, 0.25), 0.05, fill=True, facecolor='yellow', edgecolor='black'))
    ax1.text(0.5, 0.25, '+', ha='center', va='center', fontsize=14, weight='bold')
    ax1.arrow(0.5, 0.2, 0, -0.05, head_width=0.05, head_length=0.02, fc='black')
    
    # Final ReLU
    ax1.add_patch(plt.Rectangle((0.35, 0.05), 0.3, 0.1, fill=True, facecolor='lightgreen', edgecolor='black'))
    ax1.text(0.5, 0.1, 'ReLU', ha='center', fontsize=10)
    
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.axis('off')
    ax1.set_title('Basic Residual Block Architecture')
    
    # Gradient flow diagram
    ax2.text(0.5, 0.1, 'Output', ha='center', fontsize=12, weight='bold')
    ax2.arrow(0.5, 0.15, 0, 0.1, head_width=0.05, head_length=0.03, fc='blue')
    
    ax2.add_patch(plt.Rectangle((0.35, 0.3), 0.3, 0.1, fill=True, facecolor='lightgreen', edgecolor='black'))
    ax2.text(0.5, 0.35, 'ReLU', ha='center', fontsize=10)
    ax2.arrow(0.5, 0.4, 0, 0.05, head_width=0.05, head_length=0.02, fc='blue')
    
    # Gradient paths
    ax2.arrow(0.5, 0.5, 0, 0.15, head_width=0.05, head_length=0.02, fc='blue')
    ax2.text(0.55, 0.575, 'Residual\nGradient', ha='left', fontsize=9, color='blue')
    
    ax2.arrow(0.15, 0.5, 0, 0.35, head_width=0.05, head_length=0.02, fc='red', linestyle='--', linewidth=2)
    ax2.text(0.05, 0.675, 'Direct\nGradient', ha='center', fontsize=9, color='red', weight='bold')
    
    ax2.text(0.5, 0.9, 'Input', ha='center', fontsize=12, weight='bold')
    
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.axis('off')
    ax2.set_title('Gradient Flow (Backward Pass)')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Key Insights:")
    print("  ✅ Skip connection creates a 'gradient highway'")
    print("  ✅ Residual function F(x) only needs to learn small adjustments")
    print("  ✅ Identity mapping is trivial: just set F(x) = 0")
    print("  ✅ Gradients flow backward through both paths")

explore_residual_blocks()

## Part 3: Building Complete ResNet Architectures

Now let's build complete ResNet models and understand how they scale to different depths.

In [ ]:
# Compare different ResNet architectures
def compare_resnet_architectures():
    """Compare different ResNet variants."""
    
    print("🏛️ Comparing ResNet Architectures...")
    
    models = {
        'ResNet-18': ResNet18(num_classes=10),
        'ResNet-50': ResNet50(num_classes=10),
    }
    
    # Analyze each model
    print("\n📊 Architecture Comparison:")
    print("=" * 70)
    
    for name, model in models.items():
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        print(f"\n{name}:")
        print(f"  Total parameters: {total_params:,}")
        print(f"  Trainable parameters: {trainable_params:,}")
        print(f"  Model size: {total_params * 4 / 1024**2:.1f} MB (float32)")
        
        # Count residual blocks
        num_blocks = 0
        for module in model.modules():
            if isinstance(module, (BasicBlock, BottleneckBlock)):
                num_blocks += 1
        print(f"  Residual blocks: {num_blocks}")
    
    # Visualize depth comparison
    fig, ax = plt.subplots(figsize=(10, 6))
    
    model_names = list(models.keys())
    param_counts = [sum(p.numel() for p in m.parameters()) / 1e6 for m in models.values()]
    
    colors = ['#3498db', '#e74c3c']
    bars = ax.bar(model_names, param_counts, color=colors, alpha=0.7)
    
    ax.set_ylabel('Parameters (Millions)', fontsize=12)
    ax.set_title('ResNet Architecture Comparison', fontsize=14, weight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, count in zip(bars, param_counts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{count:.1f}M',
                ha='center', va='bottom', fontsize=11, weight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Test forward pass
    print("\n🧪 Testing Forward Pass:")
    test_input = torch.randn(2, 3, 224, 224).to(device)
    
    for name, model in models.items():
        model = model.to(device)
        model.eval()
        
        with torch.no_grad():
            output = model(test_input)
        
        print(f"  {name}: Input {list(test_input.shape)} → Output {list(output.shape)}")

compare_resnet_architectures()

## Part 4: Training ResNet and Observing Gradient Flow

Let's train a ResNet and monitor how gradients flow through the network during training.

In [ ]:
# Train ResNet and monitor gradient flow
def train_and_monitor_resnet():
    """Train ResNet and visualize gradient flow."""
    
    print("🎓 Training ResNet with Gradient Monitoring...")
    
    # Create smaller ResNet for faster training
    model = ResNet18(num_classes=10).to(device)
    
    # Create synthetic dataset
    print("\n📦 Creating dataset...")
    train_dataset, test_dataset = create_synthetic_dataset(
        num_classes=10,
        samples_per_class=100,
        image_size=224
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=32, shuffle=True
    )
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
    
    # Storage for metrics
    train_losses = []
    gradient_norms = []
    
    print("\n🚀 Training for 5 epochs...")
    model.train()
    
    for epoch in range(5):
        epoch_loss = 0
        epoch_grad_norms = []
        
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            
            # Collect gradient norms
            total_norm = 0
            for p in model.parameters():
                if p.grad is not None:
                    param_norm = p.grad.data.norm(2)
                    total_norm += param_norm.item() ** 2
            total_norm = total_norm ** 0.5
            epoch_grad_norms.append(total_norm)
            
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        avg_grad_norm = np.mean(epoch_grad_norms)
        
        train_losses.append(avg_loss)
        gradient_norms.append(avg_grad_norm)
        
        print(f"Epoch {epoch+1}/5: Loss = {avg_loss:.4f}, Grad Norm = {avg_grad_norm:.4f}")
    
    # Plot training progress
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    epochs = range(1, len(train_losses) + 1)
    
    # Training loss
    ax1.plot(epochs, train_losses, 'b-o', linewidth=2, markersize=8)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Training Loss', fontsize=12)
    ax1.set_title('ResNet Training Loss', fontsize=14, weight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Gradient norms
    ax2.plot(epochs, gradient_norms, 'r-s', linewidth=2, markersize=8)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Gradient Norm', fontsize=12)
    ax2.set_title('Gradient Flow Health', fontsize=14, weight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Training complete!")
    print(f"Final loss: {train_losses[-1]:.4f}")
    print(f"Gradient norms remained healthy throughout training!")
    
    return model

trained_resnet = train_and_monitor_resnet()

## Part 5: Visualizing What ResNet Learns

Let's visualize the features and activations in our trained ResNet to understand what each layer learns.

In [ ]:
# Visualize ResNet features
def visualize_resnet_features(model):
    """Visualize what ResNet layers learn."""
    
    print("👁️ Visualizing ResNet Features...")
    
    # Create a test image
    test_image = torch.randn(1, 3, 224, 224)
    
    # Add some structure
    test_image[0, 0, 80:140, 80:140] = 2.0  # Red square
    test_image[0, 1, 100:160, 100:160] = 2.0  # Green square
    test_image[0, 2, 90:150, 120:180] = 2.0  # Blue rectangle
    test_image = torch.clamp(test_image, 0, 1)
    
    # Show test image
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.imshow(test_image[0].permute(1, 2, 0))
    ax.set_title('Test Image', fontsize=14, weight='bold')
    ax.axis('off')
    plt.show()
    
    # Extract features at different layers
    model = model.cpu().eval()
    activations = {}
    
    def get_activation(name):
        def hook(module, input, output):
            activations[name] = output.detach()
        return hook
    
    # Register hooks
    model.layer1[0].register_forward_hook(get_activation('layer1'))
    model.layer2[0].register_forward_hook(get_activation('layer2'))
    model.layer3[0].register_forward_hook(get_activation('layer3'))
    model.layer4[0].register_forward_hook(get_activation('layer4'))
    
    # Forward pass
    with torch.no_grad():
        _ = model(test_image)
    
    # Visualize feature maps at different layers
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    for idx, (layer_name, activation) in enumerate(activations.items()):
        # Select first 8 channels
        num_channels = min(8, activation.shape[1])
        
        for i in range(num_channels):
            if i < 4:
                ax = axes[0, i] if idx == 0 else axes[0, i]
            else:
                ax = axes[1, i-4] if idx == 0 else axes[1, i-4]
            
            if idx == 0:  # Only plot for first layer to keep it simple
                feature_map = activation[0, i].numpy()
                im = ax.imshow(feature_map, cmap='viridis')
                ax.set_title(f'{layer_name} Ch{i}', fontsize=10)
                ax.axis('off')
    
    plt.suptitle('Feature Maps at Different ResNet Layers', fontsize=16, weight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Observations:")
    print("  • Early layers (layer1): Detect edges and simple patterns")
    print("  • Middle layers (layer2-3): Detect textures and shapes")
    print("  • Deep layers (layer4): Detect high-level features")
    print("  • Skip connections preserve information across all layers!")

visualize_resnet_features(trained_resnet)

## Part 6: ResNet vs Plain Network Comparison

Let's directly compare ResNet with skip connections against a plain network without them.

In [ ]:
# Compare ResNet vs Plain Network
def compare_resnet_vs_plain():
    """Compare learning dynamics of ResNet vs plain network."""
    
    print("⚔️ ResNet vs Plain Network Showdown...")
    
    # Create both architectures
    class PlainDeepNet(nn.Module):
        def __init__(self, num_classes=10):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 64, 7, 2, 3, bias=False)
            self.bn1 = nn.BatchNorm2d(64)
            self.maxpool = nn.MaxPool2d(3, 2, 1)
            
            # Plain layers (no skip connections)
            self.plain_layers = nn.Sequential(
                nn.Conv2d(64, 64, 3, 1, 1, bias=False), nn.BatchNorm2d(64), nn.ReLU(),
                nn.Conv2d(64, 64, 3, 1, 1, bias=False), nn.BatchNorm2d(64), nn.ReLU(),
                nn.Conv2d(64, 128, 3, 2, 1, bias=False), nn.BatchNorm2d(128), nn.ReLU(),
                nn.Conv2d(128, 128, 3, 1, 1, bias=False), nn.BatchNorm2d(128), nn.ReLU(),
            )
            
            self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
            self.fc = nn.Linear(128, num_classes)
        
        def forward(self, x):
            x = F.relu(self.bn1(self.conv1(x)))
            x = self.maxpool(x)
            x = self.plain_layers(x)
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            return self.fc(x)
    
    # Create models
    resnet_model = ResNet18(num_classes=10).to(device)
    plain_model = PlainDeepNet(num_classes=10).to(device)
    
    # Simple dataset
    X = torch.randn(500, 3, 224, 224)
    y = torch.randint(0, 10, (500,))
    dataset = torch.utils.data.TensorDataset(X, y)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Train both models
    models = {'ResNet-18': resnet_model, 'Plain Network': plain_model}
    results = {}
    
    for name, model in models.items():
        print(f"\n🏋️ Training {name}...")
        
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        criterion = nn.CrossEntropyLoss()
        
        losses = []
        grad_norms = []
        
        for epoch in range(10):
            epoch_loss = 0
            epoch_grads = []
            
            for batch_x, batch_y in dataloader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                
                # Gradient norm
                total_norm = 0
                for p in model.parameters():
                    if p.grad is not None:
                        total_norm += p.grad.data.norm(2).item() ** 2
                epoch_grads.append(total_norm ** 0.5)
                
                optimizer.step()
                epoch_loss += loss.item()
            
            avg_loss = epoch_loss / len(dataloader)
            avg_grad = np.mean(epoch_grads)
            
            losses.append(avg_loss)
            grad_norms.append(avg_grad)
            
            if (epoch + 1) % 2 == 0:
                print(f"  Epoch {epoch+1}: Loss = {avg_loss:.4f}, Grad Norm = {avg_grad:.4f}")
        
        results[name] = {'losses': losses, 'grads': grad_norms}
    
    # Plot comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = {'ResNet-18': '#2ecc71', 'Plain Network': '#e74c3c'}
    
    # Loss comparison
    for name, data in results.items():
        ax1.plot(data['losses'], label=name, linewidth=2, 
                color=colors[name], marker='o', markersize=6)
    
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Training Loss', fontsize=12)
    ax1.set_title('Training Loss: ResNet vs Plain Network', fontsize=14, weight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Gradient comparison
    for name, data in results.items():
        ax2.plot(data['grads'], label=name, linewidth=2,
                color=colors[name], marker='s', markersize=6)
    
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Gradient Norm', fontsize=12)
    ax2.set_title('Gradient Flow: ResNet vs Plain Network', fontsize=14, weight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n🏆 Results:")
    print(f"  ResNet-18 Final Loss: {results['ResNet-18']['losses'][-1]:.4f}")
    print(f"  Plain Network Final Loss: {results['Plain Network']['losses'][-1]:.4f}")
    print("\n💡 Key Takeaway:")
    print("  Skip connections enable better optimization and gradient flow!")
    print("  ResNet trains more stably and achieves better performance!")

compare_resnet_vs_plain()

## Part 7: Your Turn to Experiment!

Now it's your turn to explore ResNet! Try different modifications and experiments.

### Suggested Experiments:

1. **Depth Scaling**: Test ResNet-34, ResNet-101 and compare performance
2. **Skip Connection Ablation**: Remove skip connections and see what breaks
3. **Different Blocks**: Compare BasicBlock vs BottleneckBlock
4. **Activation Functions**: Try different activations in residual blocks
5. **Width vs Depth**: Make networks wider vs deeper

Use the cells below for your experiments!

In [ ]:
# Your experiment cell
def my_resnet_experiment():
    """Design your own ResNet experiment!"""
    
    print("🔬 Your Custom ResNet Experiment")
    
    # TODO: Design your experiment here!
    # Ideas:
    # - Test very deep networks (200+ layers)
    # - Try different skip connection patterns
    # - Experiment with block designs
    # - Compare gradient flow at different depths
    
    # Example: Test depth scaling
    depths = [18, 34, 50]
    
    print("\n📊 Testing different ResNet depths...")
    
    for depth in depths:
        if depth == 18:
            model = ResNet18(num_classes=10)
        elif depth == 50:
            model = ResNet50(num_classes=10)
        else:
            print(f"  ResNet-{depth} not implemented in this example")
            continue
        
        param_count = sum(p.numel() for p in model.parameters()) / 1e6
        print(f"  ResNet-{depth}: {param_count:.1f}M parameters")
    
    print("\n💡 Your turn: Modify this cell to create your own experiments!")

# Run your experiment
my_resnet_experiment()

## Conclusions and Takeaways

🎉 **Congratulations!** You've completed an in-depth exploration of ResNet and residual learning.

### Key Insights Discovered:

1. **Degradation Problem**: Deeper plain networks performed worse (not overfitting!)
2. **Skip Connections**: `output = F(x) + x` creates information highways
3. **Residual Learning**: Learning adjustments is easier than learning mappings
4. **Gradient Flow**: Skip connections solve vanishing gradients
5. **Scalability**: Networks can now be 100+ layers deep
6. **Universal Pattern**: Every modern architecture uses skip connections

### Why ResNet Changed Everything:

- **Enabled Depth**: Proved that deeper networks ARE better when built correctly
- **Solved Optimization**: Skip connections address both vanishing gradients and degradation
- **Universal Principle**: Inspired skip connections in transformers, GANs, and more
- **Practical Impact**: Powers most production computer vision systems today

### Modern Applications:

Every time you use:
- 📱 Phone camera features (object detection, portrait mode)
- 🚗 Autonomous vehicles (scene understanding)
- 🏥 Medical imaging (diagnosis assistance)
- 🎮 Video games (real-time graphics enhancement)
- 📺 Content recommendation (image understanding)

You're benefiting from ResNet's skip connection innovation!

### Next Steps:

1. **Explore ResNet V2**: Pre-activation improves on the original
2. **Try ResNeXt**: Grouped convolutions for efficiency
3. **Build DenseNet**: Even more connections!
4. **Apply to Tasks**: Use ResNet backbones for segmentation, detection

The skip connection revolution shows that sometimes the simplest ideas - adding just one connection - can unlock entirely new possibilities! 🚀🧠✨